# joining-tutorial.ipynb
# WESmith 11/5/22

In [ ]:
import pandas as pd
import numpy as  np
import os

### AN EXPLANATION OF MERGE() VS JOIN() AT:
### https://stackoverflow.com/questions/22676081/what-is-the-difference-between-join-and-merge-in-pandas

### "pandas.merge() is the underlying function used for all merge/join behavior."

## THE BELOW EXAMPLES FROM (BEST RESOURCE SO FAR):
### https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

## CONCAT() SECTION TO BE ENTERED ...

## MERGE()

#### SINGLE UNIQUE KEYS

In [ ]:
left = pd.DataFrame(
    {
        "key": ["K0", "K1", "K2", "K3"],
        "A": ["A0", "A1", "A2", "A3"],
        "B": ["B0", "B1", "B2", "B3"],
    }
)

In [ ]:
right = pd.DataFrame(
    {
        "key": ["K0", "K1", "K2", "K3"],
        "C": ["C0", "C1", "C2", "C3"],
        "D": ["D0", "D1", "D2", "D3"],
    }
)

In [ ]:
left

In [ ]:
right

In [ ]:
pd.merge(left, right, on="key")  # 'on' implies key is in both df's; default is inner
# both df's have the same keys, so all rows are retained

#### USING DIFFERENT KEYS (WS EXAMPLE)

In [ ]:
# WS THIS NOT USED NOW, BUT USEFUL TO SEE HOW THE COLUMNS CAN BE RENAMED
left_new = left.rename(columns={'key':'keyX'})
left_new

In [ ]:
new_right = right.copy()
new_right.loc[2,'C'] = 'K0'  # WS add a common key value into 'C' column
new_right

In [ ]:
# WS merge on two different columns
pd.merge(left, new_right, left_on='key', right_on='C')
# only the new entry in column C matches
# because 'key' is the same name in each df, merge() adds a suffice to distinguish their origin

#### MULTIPLE KEYS THAT ARE DIFFERENT

In [ ]:
# WS multiple keys
left = pd.DataFrame(
       {"key1": ["K0", "K0", "K1", "K2"],
        "key2": ["K0", "K1", "K0", "K1"],
        "A": ["A0", "A1", "A2", "A3"],
        "B": ["B0", "B1", "B2", "B3"],})
right = pd.DataFrame(
       {"key1": ["K0", "K1", "K1", "K2"],
        "key2": ["K0", "K0", "K0", "K0"],
        "C": ["C0", "C1", "C2", "C3"],
        "D": ["D0", "D1", "D2", "D3"],})

In [ ]:
left

In [ ]:
right

#### INNER MERGE (DEFAULT)

In [ ]:
result = pd.merge(left, right, on=["key1", "key2"])
result # WS has only those key pairs that exist in both frames: the intersection of the key pairs

#### LEFT FRAME KEYS MERGE

In [ ]:
result = pd.merge(left, right, how="left", on=["key1", "key2"])
result # WS has all of the key pairs from the left frame

#### RIGHT FRAME KEYS MERGE

In [ ]:
result = pd.merge(left, right, how="right", on=["key1", "key2"])
result # WS has all of the key pairss from the right frame

#### OUTER MERGE

In [ ]:
result = pd.merge(left, right, how="outer", on=["key1", "key2"])
result # WS has all key pairs in left and right frames: the union of the key pairs

#### CROSS MERGE

In [ ]:
result = pd.merge(left, right, how="cross")
result # WS like the matrix outer product: all permutations of key pairs:
# when would you want this?

#### MULTI-INDEXED SERIES MERGE WITH DATAFRAME

In [ ]:
df = pd.DataFrame({"Let": ["A", "B", "C"], "Num": [1, 2, 3]})

In [ ]:
ser = pd.Series(
    ["a", "b", "c", "d", "e", "f"],
    index=pd.MultiIndex.from_arrays(
        [["A", "B", "C"] * 2, [1, 2, 3, 4, 5, 6]], names=["Let", "Num"]))
ser

In [ ]:
df

In [ ]:
ser.reset_index()  # WS NICE WAY TO TURN A SERIES INTO A DATAFRAME

In [ ]:
df2 = pd.merge(df, ser.reset_index(), on=["Let", "Num"])  # WS default: inner merge
df2

### EXAMPLE OF MERGING WITH DUPLICATE KEYS: 
#### CAN GIVE RISE TO FRAMES THAT ARE THE MULTIPLICATION OF THE ROW DIMENSIONS

### CHECKING FOR DUPLICATE KEYS

### MERGE INDICATOR

In [ ]:
df1 = pd.DataFrame({"col1": [0, 1], "col_left": ["a", "b"]})
df1

In [ ]:
df2 = pd.DataFrame({"col1": [1, 2, 2], "col_right": [2, 2, 2]})
df2

In [ ]:
pd.merge(df1, df2, on="col1", how="outer", indicator=True)

In [ ]:
# WS can name the indicator column
pd.merge(df1, df2, on="col1", how="outer", indicator="indicator_column")

### MERGE DTYPES

#### CATEGORY DTYPES

right = pd.DataFrame(
    {"X": pd.Series(["foo", "bar"], dtype=CategoricalDtype(["foo", "bar"])),
        "Z": [1, 2],})
right

## JOINING ON INDEX AND COMPARING TO MERGING

In [ ]:
pd.merge?

In [ ]:
left = pd.DataFrame(
    {"A": ["A0", "A1", "A2"], "B": ["B0", "B1", "B2"]}, index=["K0", "K1", "K2"]
)

In [ ]:
right = pd.DataFrame(
    {"C": ["C0", "C2", "C3"], "D": ["D0", "D2", "D3"]}, index=["K0", "K2", "K3"]
)

In [ ]:
left

In [ ]:
right

### JOIN VERSION: LEFT

In [ ]:
result = left.join(right)
result  # WS all indexes on left are kept
# WS NOTE: pd.join() does not exist

In [ ]:
left.join?

#### MERGE VERSION: LEFT

In [ ]:
# WS merge gives same result as join above
# left_index and right_index must each be True, not the default False
# this means that left and right indexes are the join keys, which is the default for join()
pd.merge(left, right, how='left', left_index=True, right_index=True)

In [ ]:
# WS NOTE: merge() can be invoked as pd.merge() OR df.merge(), unlike join()
# how='left' is still needed for the left merge, even though the left df is the primary object
left.merge(right, how='left', left_index=True, right_index=True)

#### JOIN VERSION: OUTER

In [ ]:
result = left.join(right, how="outer")
result  # all indexes

#### MERGE VERSION: OUTER

In [ ]:
# WS merge gives same result as join above: indexes on both dataframes used to merge
result = pd.merge(left, right, how='outer', left_index=True, right_index=True)
result

#### JOIN VERSION: INNER

In [ ]:
result = left.join(right, how="inner")
result  # only the indexes in common

#### MERGE VERSION: INNER

In [ ]:
# WS merge gives same result as join above
result = pd.merge(left, right, how='inner', left_index=True, right_index=True)
result

## JOINING KEY COLUMNS ON AN INDEX

In [ ]:
left = pd.DataFrame(
    {
        "A": ["A0", "A1", "A2", "A3"],
        "B": ["B0", "B1", "B2", "B3"],
        "key": ["K0", "K1", "K0", "K1"],
    }
)
right = pd.DataFrame({"C": ["C0", "C1"], "D": ["D0", "D1"]}, index=["K0", "K1"])

In [ ]:
left

In [ ]:
right

#### JOIN VERSION: LEFT

In [ ]:
# ws NOTE: right df must have the keys as an INDEX for join(): see below for counter example
left.join(right, on="key")  # 'on' refers to the column of the LEFT df

In [ ]:
# WS example where the key is a column
# turning an index into a column with a desired name
wrong_right = right.reset_index().rename(columns={'index':'key'})
wrong_right

In [ ]:
# WS this works: need to create the 'key' column as an index
left.join(wrong_right.set_index('key'), on='key')

#### MERGE VERSION

In [ ]:
# WS merge gives same result as join above: 
# how = 'left' because 'left.join()' was used
# right_index = True because the index of the right table is used
# left_index  = False (default) because the left index isn't used, rather the 'key' column is used
result = pd.merge(left, right, how='left', left_on="key", right_index=True, sort=False)
result

In [ ]:
left

In [ ]:
wrong_right

In [ ]:
# WS NOTE: merge() will work with wrong_right, since there is more control with merge()
# need to specify both left and right keys: this automatically done by 'on'
pd.merge(left, wrong_right, how='left', on='key')

## MULTIPLE KEYS

In [ ]:
left = pd.DataFrame(
    {
        "A": ["A0", "A1", "A2", "A3"],
        "B": ["B0", "B1", "B2", "B3"],
        "key1": ["K0", "K0", "K1", "K2"],
        "key2": ["K0", "K1", "K0", "K1"],})

In [ ]:
index = pd.MultiIndex.from_tuples([("K0", "K0"), ("K1", "K0"), ("K2", "K0"), ("K2", "K1")])

In [ ]:
right = pd.DataFrame({"C": ["C0", "C1", "C2", "C3"], "D": ["D0", "D1", "D2", "D3"]}, index=index)

In [ ]:
left

In [ ]:
right  # WS K2 not printing in last row: likely because it is the same in the row above
# it does print if K2 in row with K1 is changed to something else

#### JOIN VERSION

In [ ]:
# WS for this to work, the right frame must have a MultiIndex (which it does here)
left.join(right, on=["key1", "key2"])  # join() uses the INDEX of the 'other' df to match

#### MERGE VERSION

In [ ]:
# WS right_index=True gives merge() the join() behavior
pd.merge(left, right, how='left', left_on=["key1", "key2"], right_index=True, sort=False)

#### JOIN VERSION: INNER

In [ ]:
result = left.join(right, on=["key1", "key2"], how="inner")
result  # WS drops rows with no match (ie NaN)

#### MERGE VERSION: INNER

In [ ]:
result = pd.merge(left, right, how='inner', left_on=["key1", "key2"], right_index=True, sort=False)
result

## JOINING A SINGLE INDEX TO A MULTIINDEX

In [ ]:
left = pd.DataFrame(
    {"A": ["A0", "A1", "A2"], "B": ["B0", "B1", "B2"]},
    index=pd.Index(["K0", "K1", "K2"], name="key"),)

In [ ]:
index = pd.MultiIndex.from_tuples(
    [("K0", "Y0"), ("K1", "Y1"), ("K2", "Y2"), ("K2", "Y3")],
    names=["key", "Y"],)
right = pd.DataFrame(
    {"C": ["C0", "C1", "C2", "C3"], "D": ["D0", "D1", "D2", "D3"]},
    index=index,)

In [ ]:
left

In [ ]:
right

#### JOIN VERSION

In [ ]:
# WS  join appears to default to the left-most index of a multiindex as the item to match
left.join(right, how="inner")

#### MERGE VERSION

In [ ]:
# tutorial says join version is faster (and of course more concise)
pd.merge(left.reset_index(), right.reset_index(), on=["key"], how="inner").set_index(["key","Y"])

In [ ]:
# break it up to examine
left

In [ ]:
left.reset_index()

In [ ]:
right

In [ ]:
right.reset_index()  # WS flattens out the multiindex

## JOINING WITH TWO MULTIINDEXES

In [ ]:
leftindex = pd.MultiIndex.from_product([list("abc"), list("xy"), [1, 2]], names=["abc", "xy", "num"])
left = pd.DataFrame({"v1": range(12)}, index=leftindex)

In [ ]:
rightindex = pd.MultiIndex.from_product([list("abc"), list("xy")], names=["abc", "xy"])
right = pd.DataFrame({"v2": [100 * i for i in range(1, 7)]}, index=rightindex)

In [ ]:
left

In [ ]:
right

In [ ]:
# WS for this to work:
# - the index for the right argument is completely used in the join, and 
# - it is a subset of the indices in the left argument; true for this example
left.join(right, on=["abc", "xy"], how="inner")

In [ ]:
# use merge if the above condition is not satisfied (another example)
leftindex = pd.MultiIndex.from_tuples(
    [("K0", "X0"), ("K0", "X1"), ("K1", "X2")], names=["key", "X"])
left = pd.DataFrame(
    {"A": ["A0", "A1", "A2"], "B": ["B0", "B1", "B2"]}, index=leftindex)

In [ ]:
rightindex = pd.MultiIndex.from_tuples(
    [("K0", "Y0"), ("K1", "Y1"), ("K2", "Y2"), ("K2", "Y3")], names=["key", "Y"])
right = pd.DataFrame(
    {"C": ["C0", "C1", "C2", "C3"], "D": ["D0", "D1", "D2", "D3"]}, index=rightindex)

In [ ]:
left

In [ ]:
right

In [ ]:
pd.merge(left.reset_index(), right.reset_index(), on=["key"], how="inner").set_index(["key", "X", "Y"])

## MERGING ON A COMBINATION OF COLUMNS AND INDEX LEVELS

In [ ]:
left_index = pd.Index(["K0", "K0", "K1", "K2"], name="key1")
left = pd.DataFrame(
       {"A": ["A0", "A1", "A2", "A3"],
        "B": ["B0", "B1", "B2", "B3"],
        "key2": ["K0", "K1", "K0", "K1"]},index=left_index)

In [ ]:
right_index = pd.Index(["K0", "K1", "K2", "K2"], name="key1")
right = pd.DataFrame(
       {"C": ["C0", "C1", "C2", "C3"],
        "D": ["D0", "D1", "D2", "D3"],
        "key2": ["K0", "K0", "K0", "K1"]},index=right_index)

In [ ]:
left

In [ ]:
right

In [ ]:
# WS comment: this is bizarre: merge() is matching both an index and a column;
# from the tutorial link:
# Strings passed as the on, left_on, and right_on parameters may refer to either 
# column names or index-level names. This enables merging DataFrame instances on a 
# combination of index levels and columns without resetting indexes.
left.merge(right, on=["key1", "key2"])

### OVERLAPPING VALUE COLUMNS

## THE BELOW EXAMPLES FROM
### see https://www.datacamp.com/tutorial/joining-dataframes-pandas

In [ ]:
dummy_data1 = {
        'id': ['1', '2', '3', '4', '5'],
        'Feature1': ['A', 'C', 'E', 'G', 'I'],
        'Feature2': ['B', 'D', 'F', 'H', 'J']}
df1 = pd.DataFrame(dummy_data1, columns = ['id', 'Feature1', 'Feature2'])
df1

In [ ]:
dummy_data2 = {
        'id': ['1', '2', '6', '7', '8'],
        'Feature1': ['K', 'M', 'O', 'Q', 'S'],
        'Feature2': ['L', 'N', 'P', 'R', 'T']}
df2 = pd.DataFrame(dummy_data2, columns = ['id', 'Feature1', 'Feature2'])
df2

In [ ]:
dummy_data3 = {
        'id': ['1', '2', '3', '4', '5', '7', '8', '9', '10', '11'],
        'Feature3': [12, 13, 14, 15, 16, 17, 15, 12, 13, 23]}
df3 = pd.DataFrame(dummy_data3, columns = ['id', 'Feature3'])
df3

## CONCAT (NOTE: CONCAT() MAKES A FULL COPY)

In [ ]:
df_row = pd.concat([df1, df2])  # WS add data sets along rows: axis=0 by default; indexes retained
df_row

In [ ]:
df_row_reindex = pd.concat([df1, df2], ignore_index=True)  # WS reset indexes
df_row_reindex

In [ ]:
frames = [df1,df2]
df_keys = pd.concat(frames, keys=['x', 'y'])  # WS keep track of where data came from
df_keys

In [ ]:
df_keys.loc['y']  #  WS retrieving a particular dataset

In [ ]:
pieces = {'x': df1, 'y': df2}  # WS label datasets in a dictionary: get the same result
df_piece = pd.concat(pieces)
df_piece

In [ ]:
df_col = pd.concat([df1,df2], axis=1)  # WS concat along columns
df_col

## MERGE

In [ ]:
df_row

In [ ]:
df3

In [ ]:
df_merge_col = pd.merge(df_row, df3, on='id')
df_merge_col
# WS if an id is not in both datasets, it is missing in merge(): ie, 6,9,10,11 missing in result
# also, IDs 1,2 are each duplicated since they were duplicated in first dataset

In [ ]:
# WS if COLUMNS on which to merge have different NAMES (unlike this case): use:
df_merge_difkey = pd.merge(df_row, df3, left_on='id', right_on='id')
df_merge_difkey

## APPEND (DEPRECATED: USE CONCAT)

In [ ]:
add_row = pd.Series(['10', 'X1', 'X2', 'X3'],
                    index=['id','Feature1', 'Feature2', 'Feature3'])
add_row

In [ ]:
# WS append is deprecated, use concat
df_add_row = df_merge_col.append(add_row, ignore_index=True)
df_add_row

In [ ]:
# WS here is the new workaround to DataFrame.append() (append deprecated because it is very slow)
# see https://stackoverflow.com/questions/70837397/
#             good-alternative-to-pandas-append-method-now-that-it-is-being-deprecated
add_row_as_df = pd.DataFrame(add_row).T  # the key is to make a DF of the Series, and transpose it
add_row_as_df

In [ ]:
# WS then can concat it
df_add_row = pd.concat([df_merge_col, add_row_as_df], ignore_index=True)
df_add_row

## JOIN

### FULL OUTER JOIN USING MERGE()

In [ ]:
df1

In [ ]:
df2

In [ ]:
# WS this automatically creates new column labels showing the source of data;
# all records from both DataFrames and fill in NaNs where data is missing
df_outer = pd.merge(df1, df2, on='id', how='outer')
df_outer

In [ ]:
# WS can rename the suffixes for the new column labels
df_suffix = pd.merge(df1, df2, left_on='id',right_on='id',how='outer',suffixes=('_left','_right'))
df_suffix

### INNER JOIN USING MERGE()

In [ ]:
df_inner = pd.merge(df1, df2, on='id', how='inner')
df_inner

### RIGHT JOIN USING MERGE()

In [ ]:
df_right = pd.merge(df1, df2, on='id', how='right')
df_right

### LEFT JOIN USING MERGE()

In [ ]:
df_left = pd.merge(df1, df2, on='id', how='left')
df_left

### JOINING ON INDEX USING MERGE()

In [ ]:
df1

In [ ]:
df2

In [ ]:
# WS this joins on the indexes (ie, here 0,1,2,3,4)
df_index = pd.merge(df1, df2, right_index=True, left_index=True)
df_index

### TIME-SERIES MERGING USING MERGE_ASOF()

In [ ]:
trades = pd.DataFrame({
    'time': pd.to_datetime(['20160525 13:30:00.023',
                            '20160525 13:30:00.038',
                            '20160525 13:30:00.048',
                            '20160525 13:30:00.048',
                            '20160525 13:30:00.048']),
    'ticker': ['MSFT', 'MSFT','GOOG', 'GOOG', 'AAPL'],
    'price': [51.95, 51.95,720.77, 720.92, 98.00],
    'quantity': [75, 155,100, 100, 100]},
    columns=['time', 'ticker', 'price', 'quantity'])
trades

In [ ]:
quotes = pd.DataFrame({
    'time': pd.to_datetime(['20160525 13:30:00.023',
                            '20160525 13:30:00.023',
                            '20160525 13:30:00.030',
                            '20160525 13:30:00.041',
                            '20160525 13:30:00.048',
                            '20160525 13:30:00.049',
                            '20160525 13:30:00.072',
                            '20160525 13:30:00.075']),
    'ticker': ['GOOG', 'MSFT', 'MSFT','MSFT', 'GOOG', 'AAPL', 'GOOG','MSFT'],
    'bid': [720.50, 51.95, 51.97, 51.99,720.50, 97.99, 720.50, 52.01],
    'ask': [720.93, 51.96, 51.98, 52.00,720.93, 98.01, 720.88, 52.03]},
    columns=['time', 'ticker', 'bid', 'ask'])
quotes

In [ ]:
df_merge_asof = pd.merge_asof(trades, quotes,
              on='time',
              by='ticker')
df_merge_asof

In [ ]:
# WS specify a time tolerance: trade must be no more than 2ms after bid/ask quote
# one MSFT thrown out because trade is 8ms after bid/ask
df_merge_asof_tolerance = pd.merge_asof(trades, quotes,
              on='time',
              by='ticker',
              tolerance=pd.Timedelta('2ms'))
df_merge_asof_tolerance

## ANOTHER SOURCE: JOIN() VS MERGE()
### from https://sparkbyexamples.com/pandas/pandas-join-vs-merge-explained-differences/

In [ ]:
technologies = {
    'Courses' :["Spark","PySpark","Python","pandas"],
    'Fee'     :[20000,25000,22000,30000],
    'Duration':['30days','40days','35days','50days'],
              }
index_labels=['r1','r2','r3','r4']
df1 = pd.DataFrame(technologies,index=index_labels)
df1

In [ ]:
technologies2 = {
    'Courses' :["Spark","Java","Python","Go"],
    'Discount':[2000,2300,1200,2000]
              }
index_labels2=['r1','r6','r3','r5']
df2 = pd.DataFrame(technologies2,index=index_labels2)
df2

In [ ]:
# pandas left join two DataFrames by Index
# WS a suffix is REQUIRED, NO DEFAULTS
df3 = df1.join(df2, how='left', lsuffix="_left", rsuffix="_right")
df3

In [ ]:
# Merge two DataFrames by index using pandas.merge()
# WS same as join above, but suffix HAS DEFAULTS
df3 = pd.merge(df1, df2, left_index=True, right_index=True, how='left')
df3

In [ ]:
# pandas merge - inner join by Column
# WS NOTE how the index column is lost from the original DataFrames
df3=pd.merge(df1,df2)
df3

In [ ]:
# Merge DataFrames by Column
# WS same result as above, inner join
df3=pd.merge(df1,df2, on='Courses')
df3

In [ ]:
# WS added
df4 = df2.rename(columns={'Courses': 'My_Courses'})
df4

In [ ]:
# When column names are different
df3=pd.merge(df1,df4, left_on='Courses', right_on='My_Courses')
df3

In [ ]:
# WS added: delete redundant column
df5 = df3.drop('My_Courses', axis=1)
df5

In [ ]:
# Use join on column
df3 = df1.set_index('Courses').join(df2.set_index('Courses'), how='inner')
df3